In [2]:
import ipywidgets
from dotenv import load_dotenv

import json
import os
from pathlib import Path
from omegaconf import OmegaConf
from pregex.core.quantifiers import Optional
from pregex.core.groups import Capture
from pregex.core.operators import Either
from pregex.meta.essentials import Integer
from pregex.core.classes import AnyLetter, AnyDigit
from pregex.core.assertions import MatchAtEnd, MatchAtStart, EnclosedBy
from pregex.core import Pregex

from typing import Literal

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

load_dotenv()

True

# Choose Season Year and Type of Game

In [3]:
def extract_game_ids_regular_or_playoff(type_of_game : Literal[1,2,3,4], path_json_files : Path):
    regex_filename = MatchAtStart(4*AnyDigit()) + str(type_of_game).zfill(2) + MatchAtEnd(4*AnyDigit())
    return list(filter(lambda p : regex_filename.get_matches(p.stem), path_json_files.glob('*')))

In [4]:

from typing import List


def extract_gamestat_regular_or_playoff(
    season_year : str,
    type_of_game : Literal[1,2,3,4]
    ) -> List[Path]:

    if os.getenv('DATA_FOLDER'):
        root_data_path = os.getenv('DATA_FOLDER')
    else:
        raise RuntimeError(f'''
                            PLEASE RUN dotenv.load_dotenv() WITH SPECIFYING A VENV VAR: DATA_FOLDER
                            ''')
    
    # The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star
    json_file_path = Path(root_data_path) / season_year 
    games_ids = extract_game_ids_regular_or_playoff(type_of_game, json_file_path)
    return games_ids


## TEST

In [5]:
YEARS = list(map(lambda p : p.name, Path('./data/').glob('[0-9]*')))
interact(
    extract_gamestat_regular_or_playoff,
    season_year = widgets.Dropdown(
        options=YEARS,
        value=YEARS[0],
        description='Choose the NHL Season year (2016 -> 2016-2017) : ',
        layout={'width': 'max-content'}
    ),
    type_of_game=widgets.ToggleButtons(
        options = [1, 2, 3, 4],
        value=3,
        description='Select type of game : ',
        tooltips=['Preseason', 'Regular season', 'Playoffs', 'All-Star'],
        layout={'width': 'max-content'}
    )
)

interactive(children=(Dropdown(description='Choose the NHL Season year (2016 -> 2016-2017) : ', layout=Layout(…

<function __main__.extract_gamestat_regular_or_playoff(season_year: str, type_of_game: Literal[1, 2, 3, 4]) -> List[pathlib.Path]>

# Choose Game ID

In [16]:
def extract_game_stat_per_games(
        data_generator : list[Path],
        game_id : int,
		verbose : bool = False,
    ):

	with open(data_generator[game_id] ,'r') as json_file_buffer:
		file_content = json.load(json_file_buffer)
	data = OmegaConf.create(file_content)

	if verbose:
		print(data.gameData.datetime.dateTime)
		print(f' {data.gameData.teams.home.abbreviation} {data.gameData.teams.home.name} (home) vs {data.gameData.teams.away.abbreviation} {data.gameData.teams.away.name} (away)')    
		print(f' \t Final Score : {data.liveData.boxscore.teams.home.teamStats.teamSkaterStats.goals} - {data.liveData.boxscore.teams.away.teamStats.teamSkaterStats.goals}')    
		print(f' \t ShotsOnGoal : {data.liveData.boxscore.teams.home.teamStats.teamSkaterStats.shots} - {data.liveData.boxscore.teams.away.teamStats.teamSkaterStats.shots}')
		print(f' \t Nb. of OTs : {len(data.liveData.linescore.periods) - 3}')
	return data

## TEST

In [17]:
interact(
    extract_game_stat_per_games,
    data_generator = fixed(extract_gamestat_regular_or_playoff(season_year='2016', type_of_game=3)),
    game_id = widgets.IntSlider(
        value=0,
        min=0,
        max=82,
        step=1,
        description='Game ID:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    ),
    verbose = fixed(True)
)

interactive(children=(IntSlider(value=0, description='Game ID:', max=82), Output()), _dom_classes=('widget-int…

<function __main__.extract_game_stat_per_games(data_generator: list[pathlib.Path], game_id: int, verbose: bool = False)>

# Choose Play ID to plot

In [18]:
def plot_play_on_svg(
        play_to_plot,
        svg_file_path,
    ):
    '''
    Using the library matplotlib and using svg_file_path as background image, 
    plot the coordinates of the play available `play_to_plot.coordinates`
    with the title of the plot `f'{play_to_plot.team.name} play - {play_to_plot.result.description} - Period {play_to_plot.about.period} {play_to_plot.about.periodTime}'`
    '''

    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    import numpy as np

    img = mpimg.imread(svg_file_path)

    # Create a figure and axis
    fig, ax = plt.subplots()
    ax.imshow(img,origin='lower')

    # Set the tick positions and labels
    ax.set_xticks([0, img.shape[1] // 2, img.shape[1]])
    ax.set_yticks([0, img.shape[0] // 2, img.shape[0]])
    ax.set_xticklabels([-img.shape[1] // 2, 0, img.shape[1] // 2])
    ax.set_yticklabels([-img.shape[0] // 2, 0, img.shape[0] // 2])

    coords = np.array(list(map(int, play_to_plot.coordinates.values()))) + np.array([img.shape[1] // 2, img.shape[0] // 2])
    ax.plot(*coords, 'go')

    plt.title(f'{play_to_plot.team.name} play - {play_to_plot.result.description} - Period {play_to_plot.about.period} {play_to_plot.about.periodTime}')
    plt.show()

In [37]:
from rich import print as rprint
from pprint import pprint as pp

In [44]:
def display_on_hockey_court_play(data_plays, index_play, svg_file_path : Path):
    '''
    Display on top of the svg file the current play coordinates available in `data_plays[index_play].`
    
    '''

    list_of_plays = data_plays.liveData.plays.allPlays
    

    play_of_interest = list_of_plays[index_play]
    if not play_of_interest.coordinates == {}:
        plot_play_on_svg(list_of_plays[index_play], svg_file_path)
    else:
        print('No coordinates available for this play')
    
    print(json.dumps(OmegaConf.to_object(play_of_interest), indent=4))

## TEST

In [43]:
interact(
    display_on_hockey_court_play,
    data_plays = fixed(extract_game_stat_per_games(
        data_generator = extract_gamestat_regular_or_playoff(season_year='2016', type_of_game=3),
        game_id = 0,
        verbose=False,
    )),
    index_play = widgets.IntSlider(
        value=0,
        min=0,
        max=len(extract_game_stat_per_games(
            data_generator = extract_gamestat_regular_or_playoff(season_year='2016', type_of_game=3),
            game_id = 0,
        ).liveData.plays.allPlays),
        step=1,
        description='Play ID:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    ),
    svg_file_path = fixed('./hocker_court.jpg')
)


interactive(children=(IntSlider(value=0, description='Play ID:', max=595), Output()), _dom_classes=('widget-in…

<function __main__.display_on_hockey_court_play(data_plays, index_play, svg_file_path: pathlib.Path)>

# Debugging Widget

In [45]:
def display_game_stat(
        season_year : str,
        type_of_game : Literal[1,2,3,4],
        game_id : int,
        index_play : int,
        SVG_FILE_PATH : str
    ):

    path_files_generator = extract_gamestat_regular_or_playoff(season_year, type_of_game)
    data_plays = extract_game_stat_per_games(path_files_generator, game_id, verbose=True)
    display_on_hockey_court_play( data_plays, index_play, SVG_FILE_PATH)

In [50]:
season_selected_widget = widgets.Dropdown(
    options=YEARS,
    value=YEARS[0],
    description="Choose the NHL Season year (2016 -> 2016-2017) : ",
    layout={"width": "max-content"},
)

type_game_selected_widget = widgets.ToggleButtons(
    options=[1, 2, 3, 4],
    value=3,
    description="Select type of game : ",
    tooltips=["Preseason", "Regular season", "Playoffs", "All-Star"],
    layout={"width": "max-content"},
)

game_id_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=1000,
    step=1,
    description="Game ID:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)

play_id_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=1000,
    step=1,
    description="Play ID:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)

SVG_FILE_PATH = "./hocker_court.jpg"

def update_game_id_widget_max(*args):
    game_id_widget.max = len(
        extract_gamestat_regular_or_playoff(
            season_selected_widget.value, type_game_selected_widget.value
        ) 
    )- 1


type_game_selected_widget.observe(update_game_id_widget_max, "value")


def update_play_id_widget_max(*args):
    play_id_widget.max = len(
        extract_game_stat_per_games(
                data_generator=extract_gamestat_regular_or_playoff(
                    season_year=season_selected_widget.value,
                    type_of_game=type_game_selected_widget.value,
                ),
                game_id=game_id_widget.value,
            ).liveData.plays.allPlays
        ) - 1


type_game_selected_widget.observe(update_play_id_widget_max, "value")

interact(
    display_game_stat,
    season_year=season_selected_widget,
    type_of_game=type_game_selected_widget,
    game_id=game_id_widget,
    index_play=play_id_widget,
    SVG_FILE_PATH=fixed(SVG_FILE_PATH),
)

interactive(children=(Dropdown(description='Choose the NHL Season year (2016 -> 2016-2017) : ', layout=Layout(…

<function __main__.display_game_stat(season_year: str, type_of_game: Literal[1, 2, 3, 4], game_id: int, index_play: int, SVG_FILE_PATH: str)>